# EMFAC - Extraction

### Imports

In [1]:
import requests
import pandas as pd
from datetime import datetime

# Local imports
from emfac_payloads import onroad_payload, offroad_payload

In [2]:
%load_ext jupyter_black

### Parameters

In [3]:
ONROAD_API = "https://arb.ca.gov/emfac/handler/request_emfac2021.php"
OFFROAD_API = "https://arb.ca.gov/emfac/handler/request_orion.php"

headers = {
    "Host": "arb.ca.gov",
    "Origin": "https://arb.ca.gov",
    "Referer": "https://arb.ca.gov/emfac/emissions-inventory/4592b9779ca87f0b0d21346bbf923961ad4d3b24",
    "Sec-Ch-Ua": '"Not.A/Brand";v="8", "Chromium";v="114", "Google Chrome";v="114"',
    "Sec-Ch-Ua-Mobile": "?0",
    "Sec-Ch-Ua-Platform": '"Windows"',
    "Sec-Fetch-Dest": "empty",
    "Sec-Fetch-Mode": "cors",
    "Sec-Fetch-Site": "same-origin",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36",
}

### Functions

In [11]:
def get_onroad_data(payload, initial_year=2015, final_year=None):
    """Get data from EMFAC API Onroad endpoint
    and parse it into a DF

    Args:
        payload (dict): Payload for EMFAC API
        initial_year (int, optional): First year of data. Defaults to 2015.
        final_year (int, optional): Last year of data. Defaults to None.

    Returns:
        df (pd.DataFrame): DataFrame with data from EMFAC API
    """

    INITIAL_MODEL_YEAR = 1965
    if final_year is None:
        final_year = 2050

    years = list(range(initial_year, final_year + 1))
    model_years = list(range(INITIAL_MODEL_YEAR, final_year + 1))
    new_payload = payload.copy()
    new_payload["form"]["calendarYear"] = years
    new_payload["form"]["calendarYearStart"] = initial_year
    new_payload["form"]["calendarYearEnd"] = final_year
    new_payload["form"]["calendarYearStartSelected"] = initial_year
    new_payload["form"]["calendarYearEndSelected"] = final_year
    new_payload["form"]["modelYear"] = model_years
    new_payload["form"]["modelYearStart"] = INITIAL_MODEL_YEAR
    new_payload["form"]["modelYearEnd"] = final_year
    new_payload["form"]["modelYearStartSelected"] = INITIAL_MODEL_YEAR
    new_payload["form"]["modelYearEndSelected"] = final_year

    response = requests.post(ONROAD_API, json=new_payload, headers=headers)
    content = response.json()
    df = pd.DataFrame(content["output"])

    return df

In [12]:
def get_offroad_data(payload, initial_year=2015, final_year=None):
    """
    Get data from EMFAC API for Offroad data

    Args:
        payload (dict): Payload for EMFAC API
        initial_year (int, optional): First year of data. Defaults to 2015.
        final_year (int, optional): Last year of data. Defaults to None.

    Returns:
        df (pd.DataFrame): DataFrame with data from EMFAC API
    """

    INITIAL_MODEL_YEAR = 1965

    if final_year is None:
        final_year = 2050

    new_payload = payload.copy()

    years = list(range(initial_year, final_year + 1))
    model_years = list(range(INITIAL_MODEL_YEAR, final_year + 1))

    new_payload = payload.copy()
    new_payload["form"]["calendarYear"] = years
    new_payload["form"]["calendarYearStart"] = initial_year
    new_payload["form"]["calendarYearEnd"] = final_year
    new_payload["form"]["calendarYearStartSelected"] = initial_year
    new_payload["form"]["calendarYearEndSelected"] = final_year
    new_payload["form"]["modelYear"] = model_years
    new_payload["form"]["modelYearStart"] = INITIAL_MODEL_YEAR
    new_payload["form"]["modelYearEnd"] = final_year
    new_payload["form"]["modelYearStartSelected"] = INITIAL_MODEL_YEAR
    new_payload["form"]["modelYearEndSelected"] = final_year

    response = requests.post(OFFROAD_API, json=payload, headers=headers)
    content = response.json()
    df = pd.DataFrame(content["output"])

    return df

### Pipeline Tests

In [13]:
df_onroad = get_onroad_data(onroad_payload, initial_year=2015, final_year=2051)

In [14]:
df_onroad.head()

,No,Region,Calendar Year,Vehicle Category,Model Year,Speed,Fuel,Population,Total VMT,CVMT,...,CO_RUNEX,CO_IDLEX,CO_STREX,CO_TOTEX,SOx_RUNEX,SOx_IDLEX,SOx_STREX,SOx_TOTEX,NH3_RUNEX,Fuel Consumption
0,1,Los Angeles (SC),2015,All Other Buses,Aggregate,Aggregate,Diesel,1953.762385962299,24609566.48576078,24609566.48576078,...,52.97565244669009,2.2035095843535166,0,55.17916203104361,0.29087872098519235,0.0039378277830101295,0,0.29481654876820246,2.3174238871031076,2781.1519128048385
1,2,Los Angeles (SC),2015,All Other Buses,Aggregate,Aggregate,Natural Gas,159.1376130264676,2755693.919855186,2755693.919855186,...,8.253545636442539,0.2503832647850134,0,8.503928901227551,0,0,0,0,3.2198896091166596,327.7037422262372
2,3,Los Angeles (SC),2015,LDA,Aggregate,Aggregate,Gasoline,3825406.0401957175,48426254296.85113,48426254296.85113,...,95882.96906828054,0,38986.229135246496,134869.19820352702,173.50044205233922,0,5.637542906678407,179.13798495901764,1435.716562923126,1910766.9575570372
3,4,Los Angeles (SC),2015,LDA,Aggregate,Aggregate,Diesel,23292.5809435353,285995884.10141146,285995884.10141146,...,135.5136826785281,0,0,135.5136826785281,0.7858738860445201,0,0,0.7858738860445201,0.9772948634670717,7408.763881707742
4,5,Los Angeles (SC),2015,LDA,Aggregate,Aggregate,Electricity,21163.715632881816,243991744.6426801,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
df_offroad = get_offroad_data(offroad_payload, initial_year=2015, final_year=2051)

In [7]:
df_offroad.head()

,No,Region,Calendar Year,Vehicle Category,Model Year,Horsepower Bin,Fuel,HC_tpd,ROG_tpd,TOG_tpd,...,NOx_tpd,CO2_tpd,PM10_tpd,PM2.5_tpd,SOx_tpd,NH3_tpd,Fuel Consumption,Total_Activity_hpy,Total_Population,Horsepower_Hours_hhpy
0,1,Los Angeles (SC),2015,Agricultural - Agricultural Tractors,Aggregate,Aggregate,Diesel,0.02522498701358821,0.030522234286441744,0.03632398129956703,...,0.19390525695294164,17.015503665209824,0.012663949637106294,0.01165083366613779,0.000021477078573382682,0.000019627883498687767,553459.3848498067,203204.6679920657,427.7073234056882,20794141.5534682
1,2,Los Angeles (SC),2015,Agricultural - Bale Wagons (Self Propelled),Aggregate,Aggregate,Diesel,0.00008213725803587057,0.00009938608222340338,0.00011827765157165361,...,0.0008330617340436496,0.0991535517294991,0.00004891387760101408,0.000045000767392932954,0.00000012709790265763448,0.00000011437653566534727,3225.1448341247406,1011.4288490069514,1.6047638202852512,120814.06990212102
2,3,Los Angeles (SC),2015,Agricultural - Balers (Self Propelled),Aggregate,Aggregate,Diesel,0.0000699738170598078,0.00008466831864236745,0.0001007622965661232,...,0.0004054754277107052,0.04043449301862543,0.00002938515132227225,0.00002703433921649047,0.00000005073575224343178,0.00000004664237591278476,1315.2034799039416,802.0862149040258,2.43468594481875,45356.25712856857
3,4,Los Angeles (SC),2015,Agricultural - Combine Harvesters,Aggregate,Aggregate,Diesel,0.0003532195888907797,0.0004273957025578436,0.0005086362080027228,...,0.0044460310795465755,0.4678580874437814,0.00018166319449331003,0.00016713013893384526,0.0000006007444889944661,0.0000005396880524342781,15217.912697058517,2986.0599461356524,7.76637276365034,641679.5525043416
4,5,Los Angeles (SC),2015,Agricultural - Construction Equipment,Aggregate,Aggregate,Diesel,0.002957662169223487,0.0035787712247604196,0.004259033523681823,...,0.025615818630963405,2.211171518254216,0.001520896117440169,0.0013992244280449564,0.0000028005650149963503,0.0000025506513242183897,71922.26879493317,29628.28773141728,57.55699442891578,3323227.4008299117
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12153,12154,Los Angeles (SC),2050,Transport Refrigeration Unit - Instate Trailer,Aggregate,Aggregate,Diesel,0.19718000000000002,2.21342,0.261773,...,2.0062000000000006,386.41638339946826,0.013750899999999998,0.012659900000000002,606627.48,0.00000866201,12568877.07,17975872.96,10042.320000000002,0
12154,12155,Los Angeles (SC),2050,Transport Refrigeration Unit - Instate Truck,Aggregate,Aggregate,Diesel,0,0,0,...,0,0,0,0,0,0,0,0,2159.92,0
12155,12156,Los Angeles (SC),2050,Transport Refrigeration Unit - Out-Of-State Ge...,Aggregate,Aggregate,Diesel,0.0065202,0.06656999999999999,0.0093884,...,0.0847073,14.126417443814793,0.00046698100000000005,0.000429643,22176.620000000003,0.000000316725,459486.74000000005,959643.8300000001,7738.869999999999,0
12156,12157,Los Angeles (SC),2050,Transport Refrigeration Unit - Out-Of-State Tr...,Aggregate,Aggregate,Diesel,0.12368599999999999,1.2501200000000003,0.15487099999999995,...,1.183954,215.3090775579221,0.0071114,0.006543150000000001,338009.4,0.00000482694,7003308.9799999995,10372981.510000002,38135.759999999995,0
